# Pipeline Simulation -  Gen

In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
from scipy.spatial.transform import Rotation as R
import os

# Mujoco Libraries
import time
import mujoco
import mujoco.viewer
from sensors import Sensors
from vectorized_point_cloud import VectorizedPC

# Open3d Libraries
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Beginning Simulation on Mujoco

In [2]:
def create_sim(xml_str, height, width, rgb=False, depth=False, segmentation=False):
    m = mujoco.MjModel.from_xml_string(xml_str)
    d = mujoco.MjData(m)

    res = [m, d]

    if rgb:
        r_rgb = mujoco.Renderer(m, height, width)
        res.append(r_rgb)

    if depth:
        r_depth = mujoco.Renderer(m, height, width)
        r_depth.enable_depth_rendering()
        res.append(r_depth)

    if segmentation:
        r_seg = mujoco.Renderer(m, height, width)
        r_seg.enable_segmentation_rendering()
        res.append(r_seg)

    return res


def new_xml(models, quat):
    assets = ""
    geoms = ""

    for model in models:
        x = (np.random.random() - 0.5)
        y = (np.random.random() - 0.5) 
        z = (np.random.random() - 0.5) + 0.395
        assets += f'<mesh name="{model}"  file="ycb/{model}" scale="1 1 1"/>\n'
        geoms += f'<geom name="{model}" pos="{x} {y} {z}" type="mesh" contype="0" conaffinity="0" group="1" density="0" mesh="{model}" quat="{quat[0]} {quat[1]} {quat[2]} {quat[3]}"/>\n'

    xml =  f"""
    <mujoco>
        <asset>
            {assets}
        </asset>
        <worldbody>
            <light name="top" pos="0 0 1"/>
            {geoms}
                        
            <camera name="camera1" pos="0 -1 0.395" euler="90 0 0" mode="fixed" fovy="60" />
            <camera name="camera2" pos="1 0 0.395"  euler="90 90 0" mode="fixed" fovy="60" />
            <camera name="camera3" pos="0 1 0.395"  euler="90 180 0" mode="fixed" fovy="60" />
            <camera name="camera4" pos="-1 0 0.395" euler="90 270 0" mode="fixed" fovy="60" />

            <camera name="camera5" pos="0.75 -0.75 0.395"  euler="90 45 0" mode="fixed" fovy="60" />
            <camera name="camera6" pos="0.75 0.75 0.395"   euler="90 135 0" mode="fixed" fovy="60" />
            <camera name="camera7" pos="-0.75 0.75 0.395"  euler="90 225 0" mode="fixed" fovy="60" />
            <camera name="camera8" pos="-0.75 -0.75 0.395" euler="90 315 0" mode="fixed" fovy="60" />
              
        </worldbody>
    </mujoco>
    """

    return xml

def show_pc(pc):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pc)
    o3d.visualization.draw_plotly([pcd])

In [3]:
# Generate xml for simulation with variables
quat = np.random.random(4)
quat = quat / np.linalg.norm(quat)
models = os.listdir('ycb')
models = np.random.choice(models, size=4)
xml = new_xml(models, quat)

# Instantiate the simulation
m, d, r_rgb, r_depth, r_seg = create_sim(xml, 480, 640, 
                                            rgb=True, 
                                            depth=True, 
                                            segmentation=True)

models

array(['056_tennis_ball.stl', '073-e_lego_duplo.stl',
       '072-b_toy_airplane.stl', '006_mustard_bottle.stl'], dtype='<U29')

In [4]:
with mujoco.viewer.launch_passive(m, d) as viewer:
# Close the viewer automatically after 30 wall-seconds.

    start = time.time()
    while viewer.is_running():
        step_start = time.time()

        # policy
        # mj_step can be replaced with code that also evaluates
        # a policy and applies a contro l signal before stepping the physics.
        mujoco.mj_step(m, d)

        # Pick up changes to the physics state, apply perturbations, update options from GUI.
        viewer.sync()

        # Rudimentary time keeping, will drift relative to wall clock.
        time_until_next_step = m.opt.timestep - (time.time() - step_start)
        if time_until_next_step > 0:
            time.sleep(time_until_next_step)
            time.sleep(0.01)

In [19]:
# Obtaining extrinsic details of the cameras
cam_num = m.cam_user.shape[0]
cam_pos = []
cam_rot = []
for i in range(cam_num):
    cam_pos.append(m.cam_pos[i])
    cam_rot.append(R.from_quat(m.cam(i).quat).as_matrix())

# Instantiating the sensors and the pointcloud
sensor = Sensors()
point_cloud = VectorizedPC((480, 640), 60)

# Making first step for the simulation
mujoco.mj_step(m, d)

# Get RGB, depth, and segmentation images
depth = [*sensor.get_depth_image_matrices(m, d, r_depth).values()]
# segmn = [*sensor.get_segment_image_matrices(m, d, r_seg).values()]

# Obtaining the pointclouds from the depth images segmented
pc = {}
mat = np.empty((0, 3))
cameras = [0,1,2,3,4,5,6,7]
for i in cameras:
    rot = (R.from_matrix(cam_rot[i]).as_euler('xyz', degrees=True)[0] - 180) * -1
    rot = R.from_euler('xyz', [rot, 0, 90], degrees=True).as_matrix()
    aux = point_cloud.get_points(depth[i],
                                 rot,
                                 cam_pos[i])
    mat = np.concatenate((mat, aux), axis=0)
    # show_pc(aux)
    # show_pc(aux)

pc_array = mat
pc_array = pc_array[np.random.choice(pc_array.shape[0], 512, replace=True), :]

np.save('sanity_check.npy', pc_array)

In [5]:
# Obtaining extrinsic details of the cameras
cam_num = m.cam_user.shape[0]
cam_pos = []
cam_rot = []
for i in range(cam_num):
    cam_pos.append(m.cam_pos[i])
    cam_rot.append(R.from_quat(m.cam(i).quat).as_matrix())

# Instantiating the sensors and the pointcloud
sensor = Sensors()
point_cloud = VectorizedPC((480, 640), 60)

# Making first step for the simulation
mujoco.mj_step(m, d)

# Get RGB, depth, and segmentation images
depth = [*sensor.get_depth_image_matrices(m, d, r_depth).values()]
segmn = [*sensor.get_segment_image_matrices(m, d, r_seg).values()]

# Obtaining the pointclouds from the depth images segmented
pc = {}
cameras = [0,1]
for i in cameras:
    rot = (R.from_matrix(cam_rot[i]).as_euler('xyz', degrees=True)[0] - 180) * -1
    rot = R.from_euler('xyz', [rot, 0, 90], degrees=True).as_matrix()
    aux = point_cloud.get_segmented_points(depth[i],
                                           segmn[i],
                                           rot,
                                           cam_pos[i])
    for key in aux.keys():
        if key == -1:
            continue
        try:
            pc[key] = np.concatenate((pc[key], aux[key]))
        except:
            pc[key] = aux[key]
    # show_pc(aux)
    # show_pc(aux)

# pc_array = mat
# pc_array = pc_array[np.random.choice(pc_array.shape[0], 512, replace=True), :]

# np.save('sanity_check.npy', pc_array)

for i in list(pc.keys()):
    show_pc(pc[i])
    print(m.geom(i).name)
    np.save(f'sanity_check_{i}.npy', pc[i])

056_tennis_ball.stl


073-e_lego_duplo.stl


072-b_toy_airplane.stl


006_mustard_bottle.stl


In [17]:
centroids = []

for i in list(pc.keys()):
    centroids.append(np.mean(pc[i], axis=0))
    
print(centroids)

complete_pc = np.empty((0, 3))
colors = np.empty((0, 3))
for i in list(pc.keys()):
    complete_pc = np.vstack((complete_pc, pc[i]))
    colors = np.vstack((colors, np.tile([0, 255, 0], (pc[i].shape[0], 1))))

for center in centroids:
    complete_pc = np.vstack((complete_pc, center))
    colors = np.vstack((colors, np.tile([255, 0, 0], (1, 1))))


pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(complete_pc)
pcd.colors = o3d.utility.Vector3dVector(colors)

o3d.visualization.draw_geometries([pcd])

[array([-0.27118469, -0.06875527,  0.4561128 ]), array([0.20522687, 0.42067906, 0.59550237]), array([0.44479695, 0.35581208, 0.86530743]), array([ 0.1264322 , -0.46349551,  0.26286566])]


## Point Cloud Visualization

In [8]:
# Displaying all pointclouds in a single 3d window
pc_array = np.load('sanity_check.npy')

pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_array)

o3d.visualization.draw_geometries([pcd])

In [22]:
# pcs = os.listdir('pcs')

# for pc in pcs:
#     print(pc)
#     show_pc(np.load('pcs/' + pc))